# MACHINE LEARING
A notebook to implement machine learning techniques.

# 1. Setup

In [322]:
# Python ≥3.5 is required
import sys
assert sys.version_info >= (3, 5)

# Disable warnings
import warnings
warnings.filterwarnings('ignore')

# Scikit-Learn ≥0.20 is required
import sklearn
assert sklearn.__version__ >= "0.20"

# Common imports
import pandas as pd
import numpy as np
import os
import copy
import json

# to make this notebook's output stable across runs
np.random.seed(42)

# To plot pretty figures
%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt
mpl.rc('axes', labelsize=14)
mpl.rc('xtick', labelsize=12)
mpl.rc('ytick', labelsize=12)
import seaborn as sns

In [323]:
# Scikit-Learn modules in use

# prepare test set
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle

# preprocessing
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.cluster import DBSCAN
from sklearn.preprocessing import PolynomialFeatures
from sklearn.impute import SimpleImputer

# train and select models
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import cross_val_score
from sklearn.metrics import r2_score
from sklearn.cluster import KMeans
from sklearn.linear_model import ElasticNet
from sklearn.base import clone
from sklearn.neighbors import KNeighborsClassifier

# cross-valiate model
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import GridSearchCV
from scipy.stats import randint, uniform

In [324]:
# Where to save the figures
PROJECT_ROOT_DIR = "."
CHAPTER_ID = "02_machine_learning"
IMAGES_PATH = os.path.join(PROJECT_ROOT_DIR, "images", CHAPTER_ID)
os.makedirs(IMAGES_PATH, exist_ok=True)

def save_fig(fig_id, tight_layout=True, fig_extension="png", resolution=300):
    path = os.path.join(IMAGES_PATH, fig_id + "." + fig_extension)
    print("Saving figure", fig_id)
    if tight_layout:
        plt.tight_layout()
    plt.savefig(path, format=fig_extension, dpi=resolution)

In [325]:
# Where to save the models
import joblib

PROJECT_ROOT_DIR = "."
CHAPTER_ID = "02_machine_learning"
MODELS_PATH = os.path.join(PROJECT_ROOT_DIR, "models", CHAPTER_ID)
os.makedirs(MODELS_PATH, exist_ok=True)

def save_model(model, model_id):
    '''To save model in the corresponding directory'''
    joblib.dump(model, os.path.join(MODELS_PATH, model_id + "." + "pkl"))

def load_model(model_id):
    '''To load model from the corresponding directory'''
    return joblib.load(os.path.join(MODELS_PATH, model_id + "." + "pkl"))

**Note:** I will jump directly to the main dishes (pipelines and metrics) since the data cleaning and EDA have been caried already.

# 2. Create a Test Set

## 2.1. Load the Data

In [326]:
# Load the dataset
df = pd.read_csv("./dataset/life_expectancy.csv")
df.head()

,Country,Year,Status,Life expectancy,Adult Mortality,infant deaths,Alcohol,percentage expenditure,Hepatitis B,Measles,...,Polio,Total expenditure,Diphtheria,HIV/AIDS,GDP,Population,thinness 1-19 years,thinness 5-9 years,Income composition of resources,Schooling
0,Afghanistan,2015,Developing,65.0,263.0,62,0.01,71.279624,65.0,1154,...,6.0,8.16,65.0,0.1,584.259210,33736494.0,17.2,17.3,0.479,10.1
1,Afghanistan,2014,Developing,59.9,271.0,64,0.01,73.523582,62.0,492,...,58.0,8.18,62.0,0.1,612.696514,327582.0,17.5,17.5,0.476,10.0
2,Afghanistan,2013,Developing,59.9,268.0,66,0.01,73.219243,64.0,430,...,62.0,8.13,64.0,0.1,631.744976,31731688.0,17.7,17.7,0.470,9.9
3,Afghanistan,2012,Developing,59.5,272.0,69,0.01,78.184215,67.0,2787,...,67.0,8.52,67.0,0.1,669.959000,3696958.0,17.9,18.0,0.463,9.8
4,Afghanistan,2011,Developing,59.2,275.0,71,0.01,7.097109,68.0,3013,...,68.0,7.87,68.0,0.1,63.537231,2978599.0,18.2,18.2,0.454,9.5


In [327]:
# Rename some columns as their names contain trailing spaces
df.rename(columns={" BMI ":"BMI","Life expectancy ":"Life_Expectancy","Adult Mortality":"Adult_Mortality",
                   "infant deaths":"Infant_Deaths","percentage expenditure":"Percentage_Exp","Hepatitis B":"HepatitisB",
                  "Measles ":"Measles"," BMI ":"BMI","under-five deaths ":"Under_Five_Deaths","Diphtheria ":"Diphtheria",
                  " HIV/AIDS":"HIV/AIDS"," thinness  1-19 years":"thinness_1to19_years"," thinness 5-9 years":"thinness_5to9_years","Income composition of resources":"Income_Comp_Of_Resources",
                   "Total expenditure":"Tot_Exp"},inplace=True)
df.columns = [alias.lower() for alias in df.columns]
df.columns

Index(['country', 'year', 'status', 'life_expectancy', 'adult_mortality',
       'infant_deaths', 'alcohol', 'percentage_exp', 'hepatitisb', 'measles',
       'bmi', 'under_five_deaths', 'polio', 'tot_exp', 'diphtheria',
       'hiv/aids', 'gdp', 'population', 'thinness_1to19_years',
       'thinness_5to9_years', 'income_comp_of_resources', 'schooling'],
      dtype='object')

In [328]:
# Drop the instances without label (life_expectancy)
df.dropna(subset=["life_expectancy"], axis=0, inplace=True)
df.life_expectancy.isnull().sum()

0

## 2.2. Create a Test Set

In [329]:
# Define the test size
test_size = 0.25

### 2.2.1. Random Sampling

In [304]:
# Create a Test Set
train_set, test_set = train_test_split(df, test_size=test_size, random_state=42)

### 2.2.2. Stratified Sampling (Optional)

**Note:** The training set and test set will preserve as much as possible the distribution of features `hiv/aids`, `income_comp_of_resources`, `schooling` and `status` as they strongly correlare with `life_expectancy` [EDA].

In [330]:
# Check the number of null values in these features
df[["hiv/aids", "income_comp_of_resources", "schooling", "status"]].isnull().sum()

hiv/aids                      0
income_comp_of_resources    160
schooling                   160
status                        0
dtype: int64

In [331]:
# Save the original dataset
df_origin = df.copy()

In [332]:
# Categorize income composition of resources (HDI)
# source: https://en.wikipedia.org/wiki/Human_Development_Index
bins = pd.IntervalIndex.from_tuples([(-0.009, 0.549), (0.549, 0.699), (0.699, 0.799), (0.799, 1)])
df["income_comp_of_resources_cat"] = pd.cut(df["income_comp_of_resources"], bins=bins)
df["income_comp_of_resources_cat"].isnull().sum()

160

In [333]:
# Categorize hiv/aids (infant deaths per 1000)
# no general criterion found, so let's categorize based on box plot in EDA
bins = pd.IntervalIndex.from_tuples([(0.099, 5.15), (5.15, 20), (20, 50.6)])
df["hiv/aids_cat"] = pd.cut(df["hiv/aids"], bins=bins)
df["hiv/aids_cat"].isnull().sum()

0

In [334]:
# Categorize schooling (number of years of schooling)
# source: https://en.wikipedia.org/wiki/Educational_stage
bins = pd.IntervalIndex.from_tuples([(-1, 3), (3, 5), (5, 12), (12, 19), (19, 21)])
df["schooling_cat"] = pd.cut(df["schooling"], bins=bins)
df["schooling_cat"].isnull().sum()

160

In [335]:
# Drop the records with missing values in the two columns
df_dropped = df[df["income_comp_of_resources"].isnull() | df["schooling"].isnull()]
df.dropna(subset=["income_comp_of_resources", "schooling"], inplace=True)
df[["income_comp_of_resources", "schooling"]].isnull().sum()

income_comp_of_resources    0
schooling                   0
dtype: int64

In [336]:
# Show the number of instances dropped
df_dropped.shape[0]

160

**Note:** Since there are just 160 rows dropped (which will not be stratified), which is around 5.5% of data, there will be little harm to the distribution of stratum if I add them back after stratified sampling.

In [337]:
# Create a test set
pre_train_set, pre_test_set = train_test_split(df, test_size=test_size, random_state=42,
                                       stratify=df[["hiv/aids_cat", "income_comp_of_resources_cat", "schooling_cat", "status"]])
pre_test_set.head()

,country,year,status,life_expectancy,adult_mortality,infant_deaths,alcohol,percentage_exp,hepatitisb,measles,...,hiv/aids,gdp,population,thinness_1to19_years,thinness_5to9_years,income_comp_of_resources,schooling,income_comp_of_resources_cat,hiv/aids_cat,schooling_cat
2456,Sri Lanka,2000,Developing,71.5,175.0,5,1.45,60.490981,NaN,16527,...,0.1,875.412178,18655.0,15.3,15.5,0.677,12.4,"(0.549, 0.699]","(0.099, 5.15]","(12, 19]"
486,Cameroon,2009,Developing,54.8,373.0,54,5.89,9.042541,8.0,251,...,6.3,123.195380,19432541.0,6.3,6.3,0.473,9.2,"(-0.009, 0.549]","(5.15, 20.0]","(5, 12]"
1518,Libya,2003,Developing,71.3,144.0,3,0.01,295.116651,96.0,0,...,0.1,4676.967530,NaN,5.6,5.4,0.740,16.0,"(0.699, 0.799]","(0.099, 5.15]","(12, 19]"
1952,Pakistan,2005,Developing,62.9,2.0,364,0.04,30.593208,7.0,2981,...,0.1,711.469946,15399667.0,21.2,21.7,0.487,6.1,"(-0.009, 0.549]","(0.099, 5.15]","(5, 12]"
65,Antigua and Barbuda,2014,Developing,76.2,131.0,0,8.56,2422.999774,99.0,0,...,0.2,12888.296670,NaN,3.3,3.3,0.782,13.9,"(0.699, 0.799]","(0.099, 5.15]","(12, 19]"


In [338]:
# Re-add the dropped rows
train_drop_set, test_drop_set = train_test_split(df_dropped, test_size=test_size, random_state=42)
train_set = pd.concat([pre_train_set, train_drop_set])
test_set = pd.concat([pre_test_set, test_drop_set])

# shuffle the sets
train_set = shuffle(train_set, random_state=42)
test_set = shuffle(test_set, random_state=42)

test_set.head() # oops! forget to drop categorized columns

,country,year,status,life_expectancy,adult_mortality,infant_deaths,alcohol,percentage_exp,hepatitisb,measles,...,hiv/aids,gdp,population,thinness_1to19_years,thinness_5to9_years,income_comp_of_resources,schooling,income_comp_of_resources_cat,hiv/aids_cat,schooling_cat
2868,Venezuela (Bolivarian Republic of),2005,Developing,73.6,158.0,9,7.92,0.000000,88.0,0,...,0.1,NaN,NaN,1.7,1.6,0.700,11.8,"(0.699, 0.799]","(0.099, 5.15]","(5, 12]"
1146,Honduras,2007,Developing,73.0,16.0,5,3.16,222.482334,93.0,0,...,0.7,1592.572182,777972.0,2.4,2.3,0.590,10.9,"(0.549, 0.699]","(0.099, 5.15]","(5, 12]"
2754,United Arab Emirates,2007,Developing,75.6,87.0,1,1.69,3759.457226,92.0,0,...,0.1,42672.613230,NaN,5.1,4.9,0.826,12.9,"(0.799, 1.0]","(0.099, 5.15]","(12, 19]"
2431,Spain,2009,Developed,81.6,66.0,2,9.99,5047.254058,96.0,41,...,0.1,32333.466100,46362946.0,0.6,0.5,0.858,16.3,"(0.799, 1.0]","(0.099, 5.15]","(12, 19]"
2165,Rwanda,2001,Developing,48.6,438.0,33,5.72,0.388254,NaN,896,...,8.1,21.569654,832946.0,7.4,7.5,0.332,7.1,"(-0.009, 0.549]","(5.15, 20.0]","(5, 12]"


In [339]:
# Drop categorized columns
train_set.drop(columns=["income_comp_of_resources_cat", "schooling_cat", "hiv/aids_cat"], inplace=True)
test_set.drop(columns=["income_comp_of_resources_cat", "schooling_cat", "hiv/aids_cat"], inplace=True)
test_set.head()

,country,year,status,life_expectancy,adult_mortality,infant_deaths,alcohol,percentage_exp,hepatitisb,measles,...,polio,tot_exp,diphtheria,hiv/aids,gdp,population,thinness_1to19_years,thinness_5to9_years,income_comp_of_resources,schooling
2868,Venezuela (Bolivarian Republic of),2005,Developing,73.6,158.0,9,7.92,0.000000,88.0,0,...,8.0,4.69,87.0,0.1,NaN,NaN,1.7,1.6,0.700,11.8
1146,Honduras,2007,Developing,73.0,16.0,5,3.16,222.482334,93.0,0,...,94.0,7.89,94.0,0.7,1592.572182,777972.0,2.4,2.3,0.590,10.9
2754,United Arab Emirates,2007,Developing,75.6,87.0,1,1.69,3759.457226,92.0,0,...,94.0,2.57,92.0,0.1,42672.613230,NaN,5.1,4.9,0.826,12.9
2431,Spain,2009,Developed,81.6,66.0,2,9.99,5047.254058,96.0,41,...,96.0,9.52,96.0,0.1,32333.466100,46362946.0,0.6,0.5,0.858,16.3
2165,Rwanda,2001,Developing,48.6,438.0,33,5.72,0.388254,NaN,896,...,76.0,4.38,77.0,8.1,21.569654,832946.0,7.4,7.5,0.332,7.1


In [340]:
# Check whether the proportions of status on the whole data and test set are close
def status_proportion(df):
    return df["status"].value_counts() / len(df)

compare_props = pd.DataFrame({
    "Overall": status_proportion(df_origin),
    "Stratified": status_proportion(test_set)
}).sort_index()

compare_props["Strat. %error"] = 100 * compare_props["Stratified"] / compare_props["Overall"] - 100
compare_props # good! :>

,Overall,Stratified,Strat. %error
Developed,0.174863,0.173497,-0.781250
Developing,0.825137,0.826503,0.165563


**Note:** Forget the `test_set` from now on! Bring it back till we find a good model.

# 3. Preprocessing

**Note:** Two categorical columns, `country`, `year` and `status`, have indices which are 0, 1 and 2. This is for the later work with `numpy` if needed.

In [341]:
# Separate features and labels
X = train_set.drop(columns=["life_expectancy"])
y = train_set["life_expectancy"].copy()

In [342]:
# Seperate numeric columns and categorical columns
num_attrbs = X.drop(["status", "country"], axis=1).columns.tolist()
cat_attrbs = ["status", "country"]
attrbs = X.columns.tolist()

**Suggestion:** May need to customize some transformers!

## 3.1. Imputer

### 3.1.1. Interpolation Imputer (Optional)

In [289]:
# Build a transformer class - interpolation imputer (by country)
class GroupInterpImputer(BaseEstimator, TransformerMixin):
    '''Transformer that treats NaN of a group using linear interpolation method
    '''
    
    def __init__(self, groupby="country",):
        self.groupby = groupby
    
    def fit(self, X, y=None): 
        return self # nothing to do
    
    
    def transform(self, X): # pd.DataFrame
        df = copy.deepcopy(X)
        
        num_attrbs = df.drop(df.columns[[0, 2]], axis=1).columns.tolist()
        
        for country in df.country.unique().tolist():
            df.loc[df[self.groupby]==country, num_attrbs] = df.loc[df[self.groupby]==country, num_attrbs].sort_values(by=["year"]).interpolate(axis=1)
            
        return df

In [290]:
# Test the new transformer
interp_imputer = GroupInterpImputer()
imputer_tester = copy.deepcopy(train_set)
interp_imputer.transform(imputer_tester).isnull().sum() # pd.DataFrame

country                     0
year                        0
status                      0
life_expectancy             0
adult_mortality             0
infant_deaths               0
alcohol                     0
percentage_exp              0
hepatitisb                  0
measles                     0
bmi                         0
under_five_deaths           0
polio                       0
tot_exp                     0
diphtheria                  0
hiv/aids                    0
gdp                         0
population                  0
thinness_1to19_years        0
thinness_5to9_years         0
income_comp_of_resources    0
schooling                   0
dtype: int64

### 3.1.2. Simple Imputers by an Atrribute

In [74]:
class AttrbBasedImputer(BaseEstimator, TransformerMixin):
    '''Impute missing values for each attribute
    '''
    
    def __init__(self, attrib="country", strategy="mean"):
        self.attrib = attrib
        self.strategy = strategy
    
    def fit(self, X, y=None):
        df = copy.deepcopy(X)
        num_attrbs = df.drop(df.columns[[0, 2]], axis=1).columns.tolist()
        
        self.statistics_ = {}
        for group in df[self.attrib].unique().tolist():
            imp_group = SimpleImputer(strategy=self.strategy)
            imp_group.fit(df.loc[df[self.attrib]==group, num_attrbs])
            self.statistics_[group] = imp_group.statistics_
        
        return self
    
    def transform(self, X):
        df = copy.deepcopy(X)
        num_attrbs = df.drop(df.columns[[0, 2]], axis=1).columns.tolist() # base
        
        for group in df[self.attrib].unique().tolist():
            df.loc[X[self.attrib]==group, num_attrbs] = self.statistics_[group]
        
        return df

**Note:** The pipeline flow is:

imputer -> feature engineer (optional) -> scaler/one-hot -> outlier detector -> clusterer (optional) -> model

## 3.2. Feature Engineering

**Note:** This section can be a large workload. However, right now just transform `country` to some relevant features or drop it.

### 3.2.1. Country to Continent

In [291]:
# Load into JSON file for later usage
with open("dataset\continent.json", "r") as f:
    country_continent = json.load(f)

country_continent

{'Afghanistan': 'Asia',
 'Albania': 'Europe',
 'Algeria': 'Africa',
 'Angola': 'Africa',
 'Antigua and Barbuda': 'Americas',
 'Argentina': 'Americas',
 'Armenia': 'Asia',
 'Australia': 'Oceania',
 'Austria': 'Europe',
 'Azerbaijan': 'Asia',
 'Bahamas': 'Americas',
 'Bahrain': 'Asia',
 'Bangladesh': 'Asia',
 'Barbados': 'Americas',
 'Belarus': 'Europe',
 'Belgium': 'Europe',
 'Belize': 'Americas',
 'Benin': 'Africa',
 'Bhutan': 'Asia',
 'Bolivia (Plurinational State of)': 'Americas',
 'Bosnia and Herzegovina': 'Europe',
 'Botswana': 'Africa',
 'Brazil': 'Americas',
 'Brunei Darussalam': 'Asia',
 'Bulgaria': 'Europe',
 'Burkina Faso': 'Africa',
 'Burundi': 'Africa',
 "Côte d'Ivoire": 'Africa',
 'Cabo Verde': 'Africa',
 'Cambodia': 'Asia',
 'Cameroon': 'Africa',
 'Canada': 'Americas',
 'Central African Republic': 'Africa',
 'Chad': 'Africa',
 'Chile': 'Americas',
 'China': 'Asia',
 'Colombia': 'Americas',
 'Comoros': 'Africa',
 'Congo': 'Africa',
 'Costa Rica': 'Americas',
 'Croatia': 'Eu

In [292]:
# Build a transformer - convert country to continent
class ContinentConverter(BaseEstimator, TransformerMixin):
    '''Transformer that replaces country with continent
    '''
    
    def __init__(self, continent=country_continent):
        self.continent = continent
    
    def fit(self, X, y=None):
        return self
        
    def transform(self, X): # np.array
        df = copy.deepcopy(X)
        for i in df.index:
            df.loc[i, "country"] = self.continent.get(df.loc[i, "country"], "Unknown")
        return df

In [294]:
# Test the new transformer
_pipeline1 = Pipeline([
        ("intern_imputer", GroupInterpImputer()),
        ("continent_converter", ContinentConverter())
    ])
X_transformed = _pipeline1.fit_transform(X)
X_transformed.isnull().sum() # no null

country                     0
year                        0
status                      0
adult_mortality             0
infant_deaths               0
alcohol                     0
percentage_exp              0
hepatitisb                  0
measles                     0
bmi                         0
under_five_deaths           0
polio                       0
tot_exp                     0
diphtheria                  0
hiv/aids                    0
gdp                         0
population                  0
thinness_1to19_years        0
thinness_5to9_years         0
income_comp_of_resources    0
schooling                   0
dtype: int64

## 3.3. Feature Scaling and Text Handling

In [295]:
# Compose scaling and one-hot encoder of the pipeline
_standardizer = ColumnTransformer([
    ("standard_scaler", StandardScaler(), num_attrbs),
    ("onehot_encoder", OneHotEncoder(), cat_attrbs)
])

_pipeline2 = Pipeline([
        ("intern_imputer", GroupInterpImputer()),
        ("continent_converter", ContinentConverter()),
        ("standardizer", _standardizer)
    ])

X_transformed = _pipeline2.fit_transform(X)
X_transformed

array([[-0.76476795,  0.40158706, -0.25556559, ...,  0.        ,
         0.        ,  0.        ],
       [-0.11180122, -0.53186527, -0.18124577, ...,  1.        ,
         0.        ,  0.        ],
       [-0.3294568 , -1.19745737, -0.23079232, ...,  0.        ,
         0.        ,  0.        ],
       ...,
       [ 1.4117878 , -0.1016655 , -0.18950353, ...,  1.        ,
         0.        ,  0.        ],
       [ 0.5411655 , -1.03511784,  0.18209561, ...,  0.        ,
         0.        ,  0.        ],
       [-0.76476795, -0.71043876, -0.23905008, ...,  0.        ,
         0.        ,  0.        ]])

## 3.4. Outlier Detection and Removal (Optional)

**Caution:** It is now disable across the system due to the unexpected negative impacts on the whole performance.

In [27]:
# Build a transformer - remove outliers using DBSCAN
class OutlierRemover(BaseEstimator, TransformerMixin):
    
    def __init__(self, eps=0.5, min_samples=5):
        self.eps = eps
        self.min_samples = min_samples
        self.dbscan = DBSCAN(eps=self.eps, min_samples=self.min_samples)
    
    def fit(self, X, y=None):
        return self
    
    def transform(self, X):
        pass # nothing else to do
    
    def fit_transform(self, X, y=None):
        y = y.to_numpy()
        self.dbscan.fit(X)
        self.labels_ = self.dbscan.labels_
        self.components_ = self.dbscan.components_
        self.core_sample_indices_ = self.dbscan.core_sample_indices_
        
        inlier_indices = [ix for ix in range(len(self.labels_)) if self.labels_[ix] != -1]
        return X[inlier_indices], y[inlier_indices]

In [28]:
# Test the new transformer
_pipeline3 = Pipeline([
        ("interp_imputer", GroupInterpImputer()),
        ("continent_converter", ContinentConverter()),
        ("standardizer", _standardizer),
        ("outlier_remover", OutlierRemover(eps=0.98, min_samples=3))
    ])

X_transformed, y_transformed = _pipeline3.fit_transform(X, y)
X_transformed

array([[-0.76476795,  0.40158706, -0.25556559, ...,  0.        ,
         0.        ,  0.        ],
       [-0.11180122, -0.53186527, -0.18124577, ...,  1.        ,
         0.        ,  0.        ],
       [-0.3294568 , -1.19745737, -0.23079232, ...,  0.        ,
         0.        ,  0.        ],
       ...,
       [ 1.19413222, -0.35329178, -0.2142768 , ...,  1.        ,
         0.        ,  0.        ],
       [ 0.5411655 , -1.03511784,  0.18209561, ...,  0.        ,
         0.        ,  0.        ],
       [-0.76476795, -0.71043876, -0.23905008, ...,  0.        ,
         0.        ,  0.        ]])

In [29]:
# Check the number of inliers
X_transformed.shape

(1691, 26)

## 3.4. Preprocessing Pipeline and Preprocessed Data

In [296]:
# Setup preprocessor pipeline
standardizer = ColumnTransformer([
    ("standard_scaler", StandardScaler(), num_attrbs),
    ("onehot_encoder", OneHotEncoder(), cat_attrbs)
])

# pre-processing pipeline for training
preprocessor = Pipeline([
        ("intern_imputer", GroupInterpImputer()),
        ("continent_converter", ContinentConverter()),
        ("standardizer", standardizer) # disable dbscan
    ])

# pre-processing pipeline for testing
test_preprocessor = Pipeline([
        ("intern_imputer", GroupInterpImputer()),
        ("continent_converter", ContinentConverter()),
        ("standardizer", standardizer)
    ])

In [297]:
# Get preprocessed data
X_prep = preprocessor.fit_transform(X)
y_prep = y
X_prep

array([[-0.76476795,  0.40158706, -0.25556559, ...,  0.        ,
         0.        ,  0.        ],
       [-0.11180122, -0.53186527, -0.18124577, ...,  1.        ,
         0.        ,  0.        ],
       [-0.3294568 , -1.19745737, -0.23079232, ...,  0.        ,
         0.        ,  0.        ],
       ...,
       [ 1.4117878 , -0.1016655 , -0.18950353, ...,  1.        ,
         0.        ,  0.        ],
       [ 0.5411655 , -1.03511784,  0.18209561, ...,  0.        ,
         0.        ,  0.        ],
       [-0.76476795, -0.71043876, -0.23905008, ...,  0.        ,
         0.        ,  0.        ]])

# 4. Select and Train Models 

**Note:** In the first approach, we only use linear-relevant models such as SVM, Linear Regression and so on.

In [298]:
# Show RMSE of a model
def root_mean_squared_error(y, y_hat):
    mse = mean_squared_error(y, y_hat)
    rmse = np.sqrt(mse)
    return rmse

In [299]:
# Show the score of cross validation
def display_scores(scores):
    print("Scores:", scores)
    print("Mean:", scores.mean())
    print("Standard deviation:", scores.std())

## 4.1. Linear Regression

In [300]:
# Train a model
lin_reg = LinearRegression()
lin_reg.fit(X_prep, y_prep)

LinearRegression()

In [301]:
# Evaluate the training
root_mean_squared_error(y_prep, lin_reg.predict(X_prep))

4.025442305434387

In [302]:
# K-fold cross validation on the model
scores = np.sqrt(-cross_val_score(lin_reg, X_prep, y_prep, scoring="neg_mean_squared_error", cv=10))
display_scores(scores)

Scores: [4.10500811 3.81847688 3.4580502  4.40698856 3.98350413 4.29550662
 3.70116329 4.23798102 4.32449396 4.27712182]
Mean: 4.0608294594022185
Standard deviation: 0.2969767396470817


## 4.2. Clustered Linear Regression

In [306]:
# Build a model of clustered Linear Regressions on clusters of K-Means
class ClusteredLinearRegression(BaseEstimator, TransformerMixin):
    
    def __init__(self, 
                 clusterer=KMeans(n_clusters=4, random_state=42), 
                 estimator=ElasticNet(alpha=0), 
                 classifier=None):
        
        self.clusterer = clusterer
        self.estimator = estimator
        self.classifier = classifier # if the clusterer cannot predict for the new instances
       
    def fit(self, X, y=None):
        # cluster the data
        self.clusterer.fit(X)
        
        # fit the data of each cluster using a elastic net
        lin_regs = {}
        clusters = np.unique(self.clusterer.labels_)
        for cluster in clusters:
            lin_reg = clone(self.estimator)
            lin_reg.fit(X[self.clusterer.labels_ == cluster], y[self.clusterer.labels_ == cluster])
            lin_regs[cluster] = lin_reg
            
        self.lin_regs_ = lin_regs
        
        # train classifier to classify the new instances (if needed)
        if self.classifier != None:
            self.classifier.fit(X, self.clusterer.labels_)
        
        return self.lin_regs_
    
    def transform(self, X):
        return self.clusterer.transform(X)
    
    def predict(self, X):
        y_hat = np.zeros(X.shape[0])
        
        # predict the cluster of new instances
        if self.classifier != None:
            clusters_pred = self.classifier.predict(X)
        else:
            clusters_pred = self.clusterer.predict(X)
        
        # predict the labels of new instances based on clusters
        for cluster in self.lin_regs_.keys():
            if X[clusters_pred == cluster].shape[0] > 0:
                y_hat_cluster = self.lin_regs_[cluster].predict(X[clusters_pred == cluster])
                y_hat[clusters_pred == cluster] = y_hat_cluster
        
        return y_hat

### 4.2.1. KMeans and Elastic Net

**Note:** The linear-based strategy is to use KMeans as the clusterer and ElasticNet (LR) as the estimator.

In [307]:
# Train a model
lin_regs = ClusteredLinearRegression(clusterer=KMeans(n_clusters=4, random_state=42), 
                                     estimator=ElasticNet(alpha=0))
lin_regs.fit(X_prep, y_prep)

{0: ElasticNet(alpha=0),
 1: ElasticNet(alpha=0),
 2: ElasticNet(alpha=0),
 3: ElasticNet(alpha=0)}

In [308]:
# Evaluate the training
root_mean_squared_error(y_prep, lin_regs.predict(X_prep))

3.5843162529828927

In [309]:
# K-fold cross validation on the model
scores = np.sqrt(-cross_val_score(lin_regs, X_prep, y_prep, scoring="neg_mean_squared_error", 
                                  cv=10, error_score="raise"))
display_scores(scores)

Scores: [ 4.19637542  4.73702828  3.12769397  4.10767808  3.56078264  4.09931368
 20.7310744   3.76792974  3.75225088  3.72440778]
Mean: 5.580453487224818
Standard deviation: 5.0665448504849255


### 4.2.2. KMeans, Elastic Net and KNN

In [310]:
# Train a model
kmeans_lins_knn = ClusteredLinearRegression(clusterer=KMeans(n_clusters=4, random_state=42), 
                                           estimator=ElasticNet(alpha=0), 
                                           classifier=KNeighborsClassifier())
kmeans_lins_knn.fit(X_prep, y_prep)

{0: ElasticNet(alpha=0),
 1: ElasticNet(alpha=0),
 2: ElasticNet(alpha=0),
 3: ElasticNet(alpha=0)}

In [311]:
# Evaluate the training
root_mean_squared_error(y_prep, kmeans_lins_knn.predict(X_prep))

3.6007485120009877

In [312]:
# K-fold cross validation on the model
scores = np.sqrt(-cross_val_score(kmeans_lins_knn, X_prep, y_prep, scoring="neg_mean_squared_error", 
                                  cv=10, error_score="raise"))
display_scores(scores)

Scores: [ 3.82858284 12.45237965  3.10203562  4.19151137  3.56682156  4.13761795
  3.4028676   3.77751546  3.93654482  3.70427372]
Mean: 4.610015059384608
Standard deviation: 2.632452617362503


# 5. Fine-tune Hyperparameters

## 5.1. Clustered Linear Regression

### 5.1.1. KMeans and Elastic Net

#### Randomized Search

In [313]:
# Perform randomized search
param_distribs = {
    "clusterer__n_clusters": randint(low=1, high=50),
    "estimator__alpha": uniform(loc=0, scale=2),
    "estimator__l1_ratio": uniform(loc=0.01, scale=0.09)
}

lins_rndsearch = RandomizedSearchCV(ClusteredLinearRegression(), param_distributions=param_distribs,
                                     n_iter=15, cv=10, scoring='neg_mean_squared_error', 
                                     error_score="raise", random_state=42)
lins_rndsearch.fit(X_prep, y_prep)

RandomizedSearchCV(cv=10, error_score='raise',
                   estimator=ClusteredLinearRegression(), n_iter=15,
                   param_distributions={'clusterer__n_clusters': <scipy.stats._distn_infrastructure.rv_frozen object at 0x00000211A76EEC40>,
                                        'estimator__alpha': <scipy.stats._distn_infrastructure.rv_frozen object at 0x00000211A778B910>,
                                        'estimator__l1_ratio': <scipy.stats._distn_infrastructure.rv_frozen object at 0x00000211A778BD60>},
                   random_state=42, scoring='neg_mean_squared_error')

In [314]:
# Evaluate score
cvres = lins_rndsearch.cv_results_
for mean_score, params in zip(cvres["mean_test_score"], cvres["params"]):
    print(np.sqrt(-mean_score), params)

4.32251854628293 {'clusterer__n_clusters': 39, 'estimator__alpha': 1.5930859737204657, 'estimator__l1_ratio': 0.02650913108795474}
4.927993530238739 {'clusterer__n_clusters': 8, 'estimator__alpha': 1.1973169683940732, 'estimator__l1_ratio': 0.024041677639819285}
3.6667983719692456 {'clusterer__n_clusters': 19, 'estimator__alpha': 0.19994983163600577, 'estimator__l1_ratio': 0.05133240027692804}
3.5610669531861006 {'clusterer__n_clusters': 36, 'estimator__alpha': 0.28573363584388156, 'estimator__l1_ratio': 0.06857996256539675}
5.245624391621732 {'clusterer__n_clusters': 2, 'estimator__alpha': 1.4439975445336495, 'estimator__l1_ratio': 0.09446974381141751}
4.4026697956033205 {'clusterer__n_clusters': 2, 'estimator__alpha': 0.36364993441420124, 'estimator__l1_ratio': 0.026506405886809047}
4.950302363482868 {'clusterer__n_clusters': 12, 'estimator__alpha': 1.2233063209765618, 'estimator__l1_ratio': 0.010635967469774566}
3.9389219372668314 {'clusterer__n_clusters': 25, 'estimator__alpha': 0.

In [315]:
# Show the best estimator
lins_rndsearch.best_params_

{'clusterer__n_clusters': 42,
 'estimator__alpha': 0.09333132642723085,
 'estimator__l1_ratio': 0.09763799669573131}

### 5.1.2. KMeans, ElasticNet and KNN

In [316]:
# Perform randomized search
param_distribs = {
    "clusterer__n_clusters": randint(low=1, high=50),
    "estimator__alpha": uniform(loc=0, scale=2),
    "estimator__l1_ratio": uniform(loc=0.01, scale=0.09),
    "classifier__n_neighbors": randint(low=2, high=15),
    "classifier__weights": ["distance"]
}

kmeans_lins_knn_rndsearch = RandomizedSearchCV(ClusteredLinearRegression(classifier=KNeighborsClassifier()), 
                                     param_distributions=param_distribs,
                                     n_iter=15, cv=10, scoring='neg_mean_squared_error', 
                                     error_score="raise", random_state=42)
kmeans_lins_knn_rndsearch.fit(X_prep, y_prep)

RandomizedSearchCV(cv=10, error_score='raise',
                   estimator=ClusteredLinearRegression(classifier=KNeighborsClassifier()),
                   n_iter=15,
                   param_distributions={'classifier__n_neighbors': <scipy.stats._distn_infrastructure.rv_frozen object at 0x00000211A778CA30>,
                                        'classifier__weights': ['distance'],
                                        'clusterer__n_clusters': <scipy.stats._distn_infrastructure.rv_frozen object at 0x00000211A76E20A0>,
                                        'estimator__alpha': <scipy.stats._distn_infrastructure.rv_frozen object at 0x00000211A76E2490>,
                                        'estimator__l1_ratio': <scipy.stats._distn_infrastructure.rv_frozen object at 0x00000211A7789610>},
                   random_state=42, scoring='neg_mean_squared_error')

In [317]:
# Evaluate score
cvres = kmeans_lins_knn_rndsearch.cv_results_
for mean_score, params in zip(cvres["mean_test_score"], cvres["params"]):
    print(np.sqrt(-mean_score), params)

3.6493872314945954 {'classifier__n_neighbors': 8, 'classifier__weights': 'distance', 'clusterer__n_clusters': 29, 'estimator__alpha': 0.3668695797323276, 'estimator__l1_ratio': 0.08017219002454923}
4.0058135643155675 {'classifier__n_neighbors': 6, 'classifier__weights': 'distance', 'clusterer__n_clusters': 39, 'estimator__alpha': 0.8916655057071823, 'estimator__l1_ratio': 0.018997742423620262}
4.062921627990668 {'classifier__n_neighbors': 12, 'classifier__weights': 'distance', 'clusterer__n_clusters': 24, 'estimator__alpha': 0.6674172222780437, 'estimator__l1_ratio': 0.022858013612974667}
3.447015274006256 {'classifier__n_neighbors': 4, 'classifier__weights': 'distance', 'clusterer__n_clusters': 22, 'estimator__alpha': 0.11282315805420051, 'estimator__l1_ratio': 0.07497988950401421}
4.457487157583792 {'classifier__n_neighbors': 7, 'classifier__weights': 'distance', 'clusterer__n_clusters': 2, 'estimator__alpha': 0.36364993441420124, 'estimator__l1_ratio': 0.026506405886809047}
3.399644

In [318]:
# Show the best params
kmeans_lins_knn_rndsearch.best_params_

{'classifier__n_neighbors': 12,
 'classifier__weights': 'distance',
 'clusterer__n_clusters': 42,
 'estimator__alpha': 0.09333132642723085,
 'estimator__l1_ratio': 0.09763799669573131}

In [319]:
# Show the best estimator
kmeans_lins_knn_rndsearch.best_estimator_

ClusteredLinearRegression(classifier=KNeighborsClassifier(n_neighbors=12,
                                                          weights='distance'),
                          clusterer=KMeans(n_clusters=42, random_state=42),
                          estimator=ElasticNet(alpha=0.09333132642723085,
                                               l1_ratio=0.09763799669573131))

# 6. Evaluate the System on the Test Set

In [343]:
# Recall test set
X_test = test_set.drop(columns=["life_expectancy"])
X_test_prep = test_preprocessor.fit_transform(X_test)
y_test = test_set["life_expectancy"].copy()

## 6.1. Clustered Linear Regression

### 6.1.1. Kmeans and Elastic Net

In [344]:
# Evaluate CLR on test set
final_models = lins_rndsearch.best_estimator_
y_pred_lrs = final_models.predict(X_test_prep)
root_mean_squared_error(y_test, y_pred_lrs)

3.2788597132218924

In [345]:
# Compute 95% confidence interval for generalization error
from scipy import stats
confidence = 0.95

squared_errors = (y_pred_lrs - y_test) ** 2
np.sqrt(stats.t.interval(confidence, len(squared_errors) - 1,
                         loc=squared_errors.mean(),
                         scale=stats.sem(squared_errors))) # t-distribution

array([2.93684427, 3.58842414])

In [346]:
# Show r2 score
r2_score(y_test, y_pred_lrs)

0.8825431934075028

### 6.1.2. Kmeans, Elastic Net and KNN

In [347]:
y_pred_lrs_knn = kmeans_lins_knn_rndsearch.best_estimator_.predict(X_test_prep)
root_mean_squared_error(y_test, y_pred_lrs_knn)

3.207499985872506

In [348]:
# Compute 95% confidence interval for generalization error
from scipy import stats
confidence = 0.95

squared_errors = (y_pred_lrs_knn - y_test) ** 2
np.sqrt(stats.t.interval(confidence, len(squared_errors) - 1,
                         loc=squared_errors.mean(),
                         scale=stats.sem(squared_errors))) # t-distribution

array([2.88934487, 3.49682693])

In [349]:
# Show r2 score
r2_score(y_test, y_pred_lrs_knn)

0.887600120920831